# Requirements

In [2]:
import os
import json
import pandas as pd
import numpy as np
import json
import requests

import re
from itertools import islice # to iterate through dicts

import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from google.oauth2 import service_account # based on google-auth library
import sddk

In [3]:
LAGT = pd.read_json("../data/large_files/LAGT_grecy_20240116.json")

In [4]:
LAGT.head(5)

,author_id,doc_id,filename,author,title,string,wordcount,source,lemmatized_sentences,lemmata_source
0,ggm0001,ggm0001.ggm001,ggm0001.ggm001.1st1K-grc1.xml,Anonymous,Anametresis Pontou,1. Χρὴ γινώσκειν ὅτι πάσης τῆς γῆς ὁ περίμετρο...,382,1Kgr,"[[Χρής, γιγνώσκω, γῆ, περίμετρος], [στάδιος, μ...",grecy
2,ogl0001,ogl0001.ogl001,ogl0001.ogl001.1st1K-grc1.xml,Pinytus,De Epistola Pinyti ad Dionysium,"FRAGMENTUM BEATI PINYTI, CNOSSI IN CRETA EPISC...",180,1Kgr,"[[Πινυτός, ἀντιγράφω, θαυμάζω, ἀποδέχω, Διονύσ...",grecy
3,stoa0033a,stoa0033a.tlg028,stoa0033a.tlg028.1st1K-grc1.xml,pseudo-Aristotle,De mundo,ΠΟΛΛΑΚΙΣ μὲν ἔμοιγε θεῖόν τι καὶ δαιμόνιον ὄντ...,6294,1Kgr,"[[θεῖος, δαιμόνιον, χρῆμα, Ἀλέξανδρος, φιλοσοφ...",grecy
4,stoa0033a,stoa0033a.tlg043,stoa0033a.tlg043.1st1K-grc1.xml,pseudo-Aristotle,De spiritu,"ΤΙΣ ἡ τοῦ ἐμφύτου πνεύματος διαμονή, καὶ τίς ἡ...",3459,1Kgr,"[[ἔμφυτος, πνεῦμα, διαμονή, αὔξησις], [Ὁρῶμεν,...",grecy
5,stoa0121,stoa0121.stoa001,stoa0121.stoa001.opp-grc1.xml,Eutropius,Breviarium historiae romanae,Τῆς Ῥωμαϊκῆς βασιλείας ἐν προοιμίοις οὐδὲν ἐγέ...,23867,1Kgr,"[[Ῥωμαϊκός, βασιλεία, προοίμιον, οὐδείς, γίγνο...",grecy


In [5]:
s = sddk.cloudSession("sciencedata.dk", "SDAM_root", "648597@au.dk")

connection with shared folder established with you as its ordinary user
endpoint variable has been configured to: https://sciencedata.dk/sharingout/648597%40au.dk/SDAM_root/


In [6]:
file_data = json.load(open("../../ServiceAccountsKey.json"))
# (2) transform the content into crendentials object
credentials = service_account.Credentials.from_service_account_info(file_data)
# (3) specify your usage of the credentials
scoped_credentials = credentials.with_scopes(
  ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive'])
# (4) use the constrained credentials for authentication of gspread package
gc = gspread.Client(auth=scoped_credentials)

AGT_overview = gc.open_by_url("https://docs.google.com/spreadsheets/d/1iVta_FuEDgUM_Lf_yByrdbbXNoVH_dnVZs6QRyYv1NM/edit?usp=sharing")
# AGT_metadata = gc.open_by_url("https://docs.google.com/spreadsheets/d/1hEUnL3E07F-EnE3wYnk1V91aXfPDrcnhFHKjD-04CM0/edit?usp=sharing")

# Extract tlg metadata from cltk github

In [7]:
tlg_authordate = requests.get("https://raw.githubusercontent.com/cltk/cltk/master/src/cltk/corpora/grc/tlg/author_date.json").json()

In [8]:
### write it out for future usage
#s.write_file("SDAM_data/OGL/tlg_authordate.json", tlg_authordate)

In [9]:
def parse_date_str(date_str):
  not_before, not_after = None, None
  if ("B.C." in date_str) & ("A.D." in date_str):
    match = re.match("([p|a]\.\s)?(\d+)?\s(B\.C\.)(\?)?(-|/)([p|a]\.\s)?(A\.D\.\s)(\d+)(\?)?", date_str)
    try:
      not_before = int(match.groups()[1]) * -100
      not_after = int(match.groups()[-2]) * 100
      if ("a. " in date_str) & ("p. " in date_str):
        not_before += 100
        not_after -= 100
    except:
      pass
  elif "B.C." in date_str:
    match = re.match("([p|a]\.\s)?(\d+)(\?)?(-|/)?(\d+)?(\?)?\s(B\.C\.)(\?)?$", date_str)
    try:
      if match.groups()[0] == "p. ":
        if match.groups()[4] != None:
          not_before = (int(match.groups()[4]) - 1)   * -100
        else:
          not_before = (int(match.groups()[1]) - 1)   * -100
      elif match.groups()[0] == "a. ":
        not_after = (int(match.groups()[1])  * -100) - 1
      else:
        not_before = int(match.groups()[1]) * -100
        if match.groups()[4] != None:
          not_after = (int(match.groups()[4]) * -100) + 99
        else:
          not_after = (int(match.groups()[1]) * -100) + 99
    except:
      pass
  else:
    match = re.match("([p|a]\.\s)?(A\.D\.\s)(\d+)(\?)?(-|/)?(\d+)?(\?)?$", date_str)
    try:
      if match.groups()[0] == "p. ":
        if match.groups()[5] != None:
          not_before = int(match.groups()[5])   * 100 +1
        else:
          not_before = int(match.groups()[2])   * 100 +1
      elif match.groups()[0] == "a. ":
        not_after = (int(match.groups()[2]) - 1)  * 100
      else:
        not_before = (int(match.groups()[2]) * 100) - 99
        if match.groups()[4] != None:
          not_after = (int(match.groups()[5]) * 100) 
        else:
          not_after = (int(match.groups()[2]) * 100)
    except:
      pass
  if not_after == 0:
    not_after = -1
  if not_before == 0:
      not_before = 1
  date_dict = {"not_before" : not_before, "not_after" : not_after}
  if "?" in date_str:
    date_dict["date_uncertain"] = True
  else:
    date_dict["date_uncertain"] = False
  return date_dict 

In [10]:
dates_dict = {}
for date_str in tlg_authordate.keys():
  dates_dict[date_str] = parse_date_str(date_str)
dates_dict

{'1 B.C.': {'not_before': -100, 'not_after': -1, 'date_uncertain': False},
 '1 B.C.-A.D. 1': {'not_before': -100,
  'not_after': 100,
  'date_uncertain': False},
 '1 B.C.-A.D. 1?': {'not_before': -100,
  'not_after': 100,
  'date_uncertain': True},
 '1 B.C./A.D. 1': {'not_before': -100,
  'not_after': 100,
  'date_uncertain': False},
 '1 B.C./A.D. 1?': {'not_before': -100,
  'not_after': 100,
  'date_uncertain': True},
 '1 B.C./A.D. 4?': {'not_before': -100,
  'not_after': 400,
  'date_uncertain': True},
 '1 B.C.?': {'not_before': -100, 'not_after': -1, 'date_uncertain': True},
 '2 B.C.': {'not_before': -200, 'not_after': -101, 'date_uncertain': False},
 '2 B.C.-A.D. 4': {'not_before': -200,
  'not_after': 400,
  'date_uncertain': False},
 '2 B.C./A.D. 2': {'not_before': -200,
  'not_after': 200,
  'date_uncertain': False},
 '2 B.C./A.D. 2?': {'not_before': -200,
  'not_after': 200,
  'date_uncertain': True},
 '2 B.C./A.D. 3': {'not_before': -200,
  'not_after': 300,
  'date_uncertain'

### Load LAGT

In [11]:
LAGT["author_id"] = LAGT.apply(lambda row: row["filename"].partition(".")[0], axis=1)

In [12]:
LAGT.head(10)

,author_id,doc_id,filename,author,title,string,wordcount,source,lemmatized_sentences,lemmata_source
0,ggm0001,ggm0001.ggm001,ggm0001.ggm001.1st1K-grc1.xml,Anonymous,Anametresis Pontou,1. Χρὴ γινώσκειν ὅτι πάσης τῆς γῆς ὁ περίμετρο...,382,1Kgr,"[[Χρής, γιγνώσκω, γῆ, περίμετρος], [στάδιος, μ...",grecy
2,ogl0001,ogl0001.ogl001,ogl0001.ogl001.1st1K-grc1.xml,Pinytus,De Epistola Pinyti ad Dionysium,"FRAGMENTUM BEATI PINYTI, CNOSSI IN CRETA EPISC...",180,1Kgr,"[[Πινυτός, ἀντιγράφω, θαυμάζω, ἀποδέχω, Διονύσ...",grecy
3,stoa0033a,stoa0033a.tlg028,stoa0033a.tlg028.1st1K-grc1.xml,pseudo-Aristotle,De mundo,ΠΟΛΛΑΚΙΣ μὲν ἔμοιγε θεῖόν τι καὶ δαιμόνιον ὄντ...,6294,1Kgr,"[[θεῖος, δαιμόνιον, χρῆμα, Ἀλέξανδρος, φιλοσοφ...",grecy
4,stoa0033a,stoa0033a.tlg043,stoa0033a.tlg043.1st1K-grc1.xml,pseudo-Aristotle,De spiritu,"ΤΙΣ ἡ τοῦ ἐμφύτου πνεύματος διαμονή, καὶ τίς ἡ...",3459,1Kgr,"[[ἔμφυτος, πνεῦμα, διαμονή, αὔξησις], [Ὁρῶμεν,...",grecy
5,stoa0121,stoa0121.stoa001,stoa0121.stoa001.opp-grc1.xml,Eutropius,Breviarium historiae romanae,Τῆς Ῥωμαϊκῆς βασιλείας ἐν προοιμίοις οὐδὲν ἐγέ...,23867,1Kgr,"[[Ῥωμαϊκός, βασιλεία, προοίμιον, οὐδείς, γίγνο...",grecy
6,stoa0146d,stoa0146d.stoa001,stoa0146d.stoa001.opp-grc1.xml,Hegemonius,Acta Archelai,Μανιχαῖος ἀπόστολος Ἰησοῦ Χριστοῦ καὶ οἱ σὺν ἐ...,5166,1Kgr,"[[Μανιχαῖος, ἀπόστολος, Ἰησοῦς, Χριστός, ἅγιος...",grecy
8,tlg0005,tlg0005.tlg003,tlg0005.tlg003.1st1K-grc1.xml,Theocritus,Syrinx,Οὐδενὸς εὐνάτειρα Μακροπτολέμοιο δὲ μάτηρ μαί...,77,1Kgr,"[[οὐδενός, εὐνητήρ], [μακροπτολέμοιο, μήτηρ, μ...",grecy
9,tlg0006,tlg0006.tlg020,tlg0006.tlg020.1st1K-grc1.xml,Euripides,Fragmenta,ποίαν σε φῶμεν γαῖαν ἐκλελοιπότα πόλει ξενοῦσθ...,17708,1Kgr,"[[φημί, γῆ, ἐκλείπω, πόλις, ξενοῦσθαι], [πάτρα...",grecy
10,tlg0007,tlg0007.tlg146,tlg0007.tlg146.1st1K-grc1.xml,Plutarch,Παροιμίαι αἷς Ἀλεξανδρεῖς ἐχρῶντο,Οἴκοι τὰ Μιλήσια: ἐπὶ τῶν ὅποι μὴ προςήκει τὴν...,2685,1Kgr,"[[Μιλήσιος], [προςήκω, τρυφή, ἐπιδείκνυμι], [Ἀ...",grecy
11,tlg0007,tlg0007.tlg147,tlg0007.tlg147.1st1K-grc1.xml,Plutarch,Ἐκλογὴ περὶ τῶν ἀδυνάτων,Κατὰ πετρῶν σπείρεις. Πλίνθον πλύνεις. Δικτύῳ ...,143,1Kgr,"[[πέτρα, σπείρω], [Πλίνθος, πλύνω, Δίκτυον, ἄν...",grecy


### Map TLG dates on LAGT

In [13]:
tlg_authordate_reversed = {}
for key, val in tlg_authordate.items():
  for id in val:
    tlg_authordate_reversed["tlg" + id] = key 

In [14]:
def get_date(author_id):
  try:
    return tlg_authordate_reversed[author_id]
  except:
    return None
LAGT["tlg_date"] = LAGT["author_id"].apply(get_date)

In [15]:
for col in ["not_before", "not_after", "date_uncertain"]:
  LAGT[col] = LAGT["tlg_date"].apply(lambda x: dates_dict[x][col] if x is not None else None)

In [16]:
LAGT.head(10)

,author_id,doc_id,filename,author,title,string,wordcount,source,lemmatized_sentences,lemmata_source,tlg_date,not_before,not_after,date_uncertain
0,ggm0001,ggm0001.ggm001,ggm0001.ggm001.1st1K-grc1.xml,Anonymous,Anametresis Pontou,1. Χρὴ γινώσκειν ὅτι πάσης τῆς γῆς ὁ περίμετρο...,382,1Kgr,"[[Χρής, γιγνώσκω, γῆ, περίμετρος], [στάδιος, μ...",grecy,None,NaN,NaN,None
2,ogl0001,ogl0001.ogl001,ogl0001.ogl001.1st1K-grc1.xml,Pinytus,De Epistola Pinyti ad Dionysium,"FRAGMENTUM BEATI PINYTI, CNOSSI IN CRETA EPISC...",180,1Kgr,"[[Πινυτός, ἀντιγράφω, θαυμάζω, ἀποδέχω, Διονύσ...",grecy,None,NaN,NaN,None
3,stoa0033a,stoa0033a.tlg028,stoa0033a.tlg028.1st1K-grc1.xml,pseudo-Aristotle,De mundo,ΠΟΛΛΑΚΙΣ μὲν ἔμοιγε θεῖόν τι καὶ δαιμόνιον ὄντ...,6294,1Kgr,"[[θεῖος, δαιμόνιον, χρῆμα, Ἀλέξανδρος, φιλοσοφ...",grecy,None,NaN,NaN,None
4,stoa0033a,stoa0033a.tlg043,stoa0033a.tlg043.1st1K-grc1.xml,pseudo-Aristotle,De spiritu,"ΤΙΣ ἡ τοῦ ἐμφύτου πνεύματος διαμονή, καὶ τίς ἡ...",3459,1Kgr,"[[ἔμφυτος, πνεῦμα, διαμονή, αὔξησις], [Ὁρῶμεν,...",grecy,None,NaN,NaN,None
5,stoa0121,stoa0121.stoa001,stoa0121.stoa001.opp-grc1.xml,Eutropius,Breviarium historiae romanae,Τῆς Ῥωμαϊκῆς βασιλείας ἐν προοιμίοις οὐδὲν ἐγέ...,23867,1Kgr,"[[Ῥωμαϊκός, βασιλεία, προοίμιον, οὐδείς, γίγνο...",grecy,None,NaN,NaN,None
6,stoa0146d,stoa0146d.stoa001,stoa0146d.stoa001.opp-grc1.xml,Hegemonius,Acta Archelai,Μανιχαῖος ἀπόστολος Ἰησοῦ Χριστοῦ καὶ οἱ σὺν ἐ...,5166,1Kgr,"[[Μανιχαῖος, ἀπόστολος, Ἰησοῦς, Χριστός, ἅγιος...",grecy,None,NaN,NaN,None
8,tlg0005,tlg0005.tlg003,tlg0005.tlg003.1st1K-grc1.xml,Theocritus,Syrinx,Οὐδενὸς εὐνάτειρα Μακροπτολέμοιο δὲ μάτηρ μαί...,77,1Kgr,"[[οὐδενός, εὐνητήρ], [μακροπτολέμοιο, μήτηρ, μ...",grecy,4-3 B.C.,-400.0,-201.0,False
9,tlg0006,tlg0006.tlg020,tlg0006.tlg020.1st1K-grc1.xml,Euripides,Fragmenta,ποίαν σε φῶμεν γαῖαν ἐκλελοιπότα πόλει ξενοῦσθ...,17708,1Kgr,"[[φημί, γῆ, ἐκλείπω, πόλις, ξενοῦσθαι], [πάτρα...",grecy,5 B.C.,-500.0,-401.0,False
10,tlg0007,tlg0007.tlg146,tlg0007.tlg146.1st1K-grc1.xml,Plutarch,Παροιμίαι αἷς Ἀλεξανδρεῖς ἐχρῶντο,Οἴκοι τὰ Μιλήσια: ἐπὶ τῶν ὅποι μὴ προςήκει τὴν...,2685,1Kgr,"[[Μιλήσιος], [προςήκω, τρυφή, ἐπιδείκνυμι], [Ἀ...",grecy,A.D. 1-2,1.0,200.0,False
11,tlg0007,tlg0007.tlg147,tlg0007.tlg147.1st1K-grc1.xml,Plutarch,Ἐκλογὴ περὶ τῶν ἀδυνάτων,Κατὰ πετρῶν σπείρεις. Πλίνθον πλύνεις. Δικτύῳ ...,143,1Kgr,"[[πέτρα, σπείρω], [Πλίνθος, πλύνω, Δίκτυον, ἄν...",grecy,A.D. 1-2,1.0,200.0,False


# Add TLG epithets

In [17]:
resp = requests.get("https://raw.githubusercontent.com/cltk/cltk/master/src/cltk/corpora/grc/tlg/author_epithet.json")
tlg_author_epithet = json.loads(resp.content)
tlg_author_epithet

{'Alchemistae': ['1016',
  '2019',
  '2140',
  '2181',
  '2589',
  '2632',
  '4086',
  '4319',
  '4323',
  '4324',
  '4325',
  '4326',
  '4327',
  '4328',
  '4329',
  '4330',
  '4331',
  '4332',
  '4333',
  '4334',
  '4335',
  '4337',
  '4338',
  '4339',
  '4340',
  '9021'],
 'Apologetici': ['0531',
  '0645',
  '1163',
  '1184',
  '1205',
  '1495',
  '1725',
  '1766',
  '1992'],
 'Astrologici': ['1004',
  '1105',
  '1222',
  '1337',
  '1487',
  '1703',
  '1764',
  '2043',
  '2053',
  '2583',
  '3014',
  '3047',
  '4361',
  '4374'],
 'Astronomici': ['0653',
  '0717',
  '1181',
  '1207',
  '1210',
  '1272',
  '1358',
  '1383',
  '1393',
  '1431',
  '1719',
  '2133',
  '2644',
  '3014'],
 'Atticistae': ['1323', '1515', '1569', '1608'],
 'Biographi': ['0004',
  '0007',
  '0608',
  '1167',
  '2585',
  '2770',
  '2877',
  '3115',
  '4282',
  '9012'],
 'Bucolici': ['0005', '0035', '0036'],
 'Choliambographi': ['1257'],
 'Chronographi': ['2871', '3018', '3043', '3045', '3140', '4046'],
 'Comic

In [18]:
tlg_author_epithet_reversed = {}
for key in tlg_author_epithet.keys():
    for author_id in tlg_author_epithet[key]:
        try:
          tlg_author_epithet_reversed["tlg" + author_id] = tlg_author_epithet_reversed["tlg" + author_id] + [key]
        except:
          tlg_author_epithet_reversed["tlg" + author_id] = [key]
        
def get_epithet(author_id): # we use doc_id, since author id was modified
    try:
        epithets = tlg_author_epithet_reversed[author_id[:7]]
    except:
        epithets = []
    return epithets

LAGT["tlg_epithet"] = LAGT["author_id"].apply(get_epithet)

In [36]:
### Add TLG locations

In [ ]:
LAGT["lemmatacount"]

In [21]:
s.write_file("SDAM_data/AGT/LAGT_dated_20240129.json", LAGT)

A file with the same name ("LAGT_dated_20240129.json") already exists in this location.
Your <class 'pandas.core.frame.DataFrame'> object has been succesfully written as "https://sciencedata.dk/sharingout/648597%40au.dk/SDAM_root/SDAM_data/AGT/LAGT_dated_20240129.json"
